In [2]:
%reset

In [3]:
import numpy as np
import wandb
import torch
import pcl
import open3d as o3d
import time

filepath = "/home/server/PIN_SLAM/data/wudasuidao/normal/30frame/1.ply"
pc_load = o3d.io.read_point_cloud(filepath)
# origin_points = np.asarray(pc_load.points)

# pc_load.paint_uniform_color([0, 0, 1.0])  # 渲染颜色
point1 = np.asarray(pc_load.points)
normals = np.asarray(pc_load.normals)


# normals = np.asarray(pc_load.normals)

o3d.visualization.draw_geometries([pc_load],
                                # zoom=0.3412,
                                # front=[0.4257, -0.2125, -0.8795],
                                # lookat=[2.6172, 2.0475, 1.532],
                                # up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=True
                                )

In [41]:
o3d.visualization.draw_geometries([pc_load],
#                                 zoom=0.3412,
#                                 front=[0.4257, -0.2125, -0.8795],
#                                 lookat=[2.6172, 2.0475, 1.532],
#                                 up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=True
                                )

# 法向量估计（knn + radius）

In [4]:
# 剔除近邻点少的点，相当于作了一次radius剔除
# parameters
k_out = 30
radius_out = 3
pc_load, ind = pc_load.remove_radius_outlier(nb_points=k_out, radius=radius_out) # cl是pointcloud ind是剩下的点的索引
# pc_load, idx = pc_load.remove_statistical_outlier(nb_neighbors=k, std_ratio=3, print_progress=False)


In [8]:
k = 20
radius = 0.3
pc_load.estimate_normals(
    # search_param=o3d.geometry.KDTreeSearchParamHybrid(radius = radius, max_nn = k)
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn = k)
)
# pc_load.orient_normals_towards_camera_location(camera_location=np.array([0.0, 0.0, 0.0]))
neighbors_normal_index={}

## neighbors_normal_index
kdtree = o3d.geometry.KDTreeFlann(pc_load)

T1 = time.time()
origin_points = np.asarray(pc_load.points)

# Only costs 1.11s-> 0.3s
for i in range(len(pc_load.points)):
    [a,idx,b] = kdtree.search_knn_vector_3d(pc_load.points[i], k)
    neighbors_normal_index[i] = idx

# kk = neighbors_normal_index[1][0]

T2 = time.time()
print(f"Time cost:{T2-T1}")

normals = np.asarray(pc_load.normals)
norms = np.linalg.norm(normals, axis=1, keepdims=True)
# 归一化
normals = normals / norms
old_normals = np.copy(normals)

# pc_load.orient_normals_towards_camera_location()
pc_load.orient_normals_consistent_tangent_plane(100)


o3d.visualization.draw_geometries([pc_load],
                                # zoom=0.3412,
                                # front=[0.4257, -0.2125, -0.8795],
                                # lookat=[2.6172, 2.0475, 1.532],
                                # up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=True)

Time cost:0.48326897621154785


AttributeError: module 'open3d.cuda.pybind.geometry' has no attribute 'orient_normals_to_align_with_direction'

# 法向量优化 myself

In [34]:
n_hat = np.copy(normals)
normals_temp = np.copy(normals)
[point_num, dimension] = normals.shape

diff_op = np.zeros((k * point_num, dimension))
sita = np.ones((k * point_num, dimension))
beta = 0.05 # 0.01
nita = 0.10 # 0.05
threshold = 0.1
iternum = 0

while np.max(np.abs(diff_op - sita)) > threshold:
    iternum += 1
    for i in range(point_num):
        neighbor_indices = np.array(neighbors_normal_index[i])  # 获取第i个点的邻居索引
        neighbor_normals = normals_temp[neighbor_indices]  # 直接获取所有邻居的法向量
        diffs = normals_temp[i] - neighbor_normals  # 一次计算所有差值
        diff_op[i*k:(i+1)*k] = diffs
        
        bb = np.sum(diffs**2, axis=1)
        mask = (nita/beta > bb)
        sita[i*k:(i+1)*k][mask] = 0
        sita[i*k:(i+1)*k][~mask] = diffs[~mask]
        
        # 计算新的 normals[i] 值
        normals[i] = (n_hat[i] + beta * np.sum(neighbor_normals + sita[i*k:(i+1)*k], axis=0)) / (1 + beta * k)
    
    normals_temp = np.copy(normals)


    print(f"The iteration number is {iternum}, the biggest diff is {np.max(np.abs(diff_op - sita))}, beta is {beta}")
    beta *= 2

norms = np.linalg.norm(normals, axis=1, keepdims=True)
normals /= norms  # 归一化


The iteration number is 1, the biggest diff is 1.341410664754473, beta is 0.05
The iteration number is 2, the biggest diff is 0.9477380393092849, beta is 0.1
The iteration number is 3, the biggest diff is 0.6775046570225257, beta is 0.2
The iteration number is 4, the biggest diff is 0.48338427162227715, beta is 0.4
The iteration number is 5, the biggest diff is 0.3513067861103475, beta is 0.8
The iteration number is 6, the biggest diff is 0.24682163875800778, beta is 1.6
The iteration number is 7, the biggest diff is 0.17536532505462968, beta is 3.2
The iteration number is 8, the biggest diff is 0.1233723286857707, beta is 6.4
The iteration number is 9, the biggest diff is 0.08745144965980622, beta is 12.8


In [9]:
pc_load.normals = o3d.utility.Vector3dVector(normals)

o3d.visualization.draw_geometries([pc_load],
                                zoom=0.3412,
                                front=[0.4257, -0.2125, -0.8795],
                                lookat=[2.6172, 2.0475, 1.532],
                                up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=True)


In [11]:
array = np.hstack((pc_load.points, pc_load.normals))

# PSS-BA least square

In [ ]:
## step2: 构造optimization function
n_hat = np.copy(normals)
# iter_value = normals
[point_num, dimesion] = normals.shape
diff_op = np.zeros((k*point_num, 1))
sita = np.ones((k*point_num, 1))
beta = 0.05
nita = 0.08 # 0.05-0.1
threshold = 0.01
iternum = 0
aa = np.max(diff_op - sita)

B = np.zeros((k,3))
f = np.ones((k,1))

while(np.abs(np.max(diff_op - sita)) > threshold):
    iternum = iternum + 1
    ## 第一个优化的解
    for i in range(point_num):
        for j in range(k):
            diff_op[i*k+j] = 1 - np.dot(normals[i,:].T,normals[neighbors_normal_index[i][j],:])
            bb = np.square(diff_op[i*k+j])
            if (nita/beta > bb):
                sita[i*k+j] = 0
            else:
                sita[i*k+j] = np.copy(diff_op[i*k+j])

            B = -n_hat[i,:].reshape(-1,3) - beta * n_hat[neighbors_normal_index[i],:] # (1,3)-(k,3)
            f[j,:] = 1
        # 计算新的 normals[i] 值

        normals[i] = ((np.linalg.inv(B.T @ B)) @ B.T @ f).T
            

    print(f"The iteration number is {iternum} , the biggest diff is {np.abs(np.max(diff_op - sita))} , beta is {beta}")
    beta = 2 * beta

norms = np.linalg.norm(normals, axis=1, keepdims=True)
# 归一化
normals = normals / norms

The iteration number is 1 , the biggest diff is 1.264900210017501 , beta is 0.05
The iteration number is 2 , the biggest diff is 0.8943048580169232 , beta is 0.1
The iteration number is 3 , the biggest diff is 0.6324056548035415 , beta is 0.2
The iteration number is 4 , the biggest diff is 0.44717045329851235 , beta is 0.4
The iteration number is 5 , the biggest diff is 0.3161800464932658 , beta is 0.8
The iteration number is 6 , the biggest diff is 0.2235375429060713 , beta is 1.6
The iteration number is 7 , the biggest diff is 0.15800779126912823 , beta is 3.2
The iteration number is 8 , the biggest diff is 0.11178282515574134 , beta is 6.4
The iteration number is 9 , the biggest diff is 0.07900266442162951 , beta is 12.8
The iteration number is 10 , the biggest diff is 0.05585170018030283 , beta is 25.6
The iteration number is 11 , the biggest diff is 0.03946204616707005 , beta is 51.2
The iteration number is 12 , the biggest diff is 0.0277789158675108 , beta is 102.4
The iteration 

# 点降噪

In [37]:
p_hat = np.copy(origin_points)
points_iter = np.copy(origin_points)
points_temp = np.copy(origin_points)
normals # 优化后法向量
k = 10

neighbors_point={}
for i in range(len(origin_points)):
    [_,idx,_] = kdtree.search_knn_vector_3d(pc_load.points[i], k)
    neighbors_point[i] = idx

[point_num, dimesion] = origin_points.shape
# diff_op_point = np.zeros((k*point_num, dimesion))
kesai = np.ones((k*point_num, 1)) # 辅助变量
DP_ikj = np.zeros((k*point_num, 1)) # 新的DP
DP_ikj_pre = np.zeros((k*point_num, 1)) # 上一次迭代的，即DP～
alpha = np.eye(point_num, dtype=float) # alpha对角阵
delta = 0.005 # 0.002-0.008
t = 0.008 # 辅助变量系数
threshold = 0.01
iternum = 0
aa = np.max(diff_op - sita)

while(np.abs(np.max(DP_ikj - kesai)) > threshold):
    iternum = iternum + 1
    ## 第一个优化的解
    for i in range(point_num):
        DP_ikj_pre = np.copy(DP_ikj) #存储上一次的
        for j in range(k):
            # diff_op_point[i*k+j,:] = (points_iter[i,:] - points_iter[neighbors_point[i][j],:])*normals[i,:]

            # 这一次的Dp
            DP_ikj[i*k+j] = np.dot(normals[i,:] ,(points_iter[i,:] - points_iter[neighbors_point[i][j],:]))
            if (t/delta > np.square(DP_ikj[i*k+j])):
                kesai[i*k+j] = 0
            else:
                kesai[i*k+j] = np.copy(DP_ikj[i*k+j])
        
        # for j in range(k):
        #     alpha[i,i] = ((p_hat[i,:] + delta * ((points_iter[neighbors_point[i][j],:] + alpha[neighbors_point[i][j],neighbors_point[i][j]] * normals[neighbors_point[i][j],:])*normals[i,:] + kesai[i*k+j,:])) / (1 + delta * normals[i,:]) - points_iter[i,:]) / normals[i,:]

        # DP_ikj_pre = np.copy(DP_ikj) #存储上一次的

        for j in range(k):
            term1 = np.dot((points_temp[i,:] - p_hat[i,:]) , normals[i,:])
            term2 = delta * (DP_ikj_pre[i*k+j,:] - np.dot(normals[i,:],normals[neighbors_point[i][j],:])*alpha[neighbors_point[i][j],neighbors_point[i][j]] - kesai[i*k+j])
            term3 = np.dot(normals[i,:], normals[i,:]) + delta
            alpha[i,i] = -1 * (term1+term2) / term3
        
        points_iter[i,:] = points_temp[i,:] + alpha[i,i] * normals[i,:]
    points_temp = np.copy(points_iter)

    print(f"The iteration number is {iternum} , the biggest diff is {np.abs(np.max(DP_ikj - kesai))} , delta is {delta}")
    delta = 2 * delta


The iteration number is 1 , the biggest diff is 1.2646627174738052 , delta is 0.005
The iteration number is 2 , the biggest diff is 0.8942334088408712 , delta is 0.01
The iteration number is 3 , the biggest diff is 0.6231808104524139 , delta is 0.02
The iteration number is 4 , the biggest diff is 0.4466751658616845 , delta is 0.04
The iteration number is 5 , the biggest diff is 0.3157877811830476 , delta is 0.08
The iteration number is 6 , the biggest diff is 0.22344069747220302 , delta is 0.16
The iteration number is 7 , the biggest diff is 0.15808646560787723 , delta is 0.32
The iteration number is 8 , the biggest diff is 0.11178110088097201 , delta is 0.64
The iteration number is 9 , the biggest diff is 0.07894166771829043 , delta is 1.28
The iteration number is 10 , the biggest diff is 0.05588982705518825 , delta is 2.56
The iteration number is 11 , the biggest diff is 0.039526798219989466 , delta is 5.12
The iteration number is 12 , the biggest diff is 0.02793880965217967 , delta 

In [42]:
pcd=o3d.geometry.PointCloud()
pcd.points= o3d.utility.Vector3dVector(points_iter)
pcd.normals = o3d.utility.Vector3dVector(normals)
o3d.visualization.draw_geometries([pcd],
                                zoom=0.3412,
                                front=[0.4257, -0.2125, -0.8795],
                                lookat=[2.6172, 2.0475, 1.532],
                                up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=False)

## neighbors_normal_index

In [ ]:
# This method use 13.76s
# for i in range(len(origin_points)):
#     # 对于每个点，找到其k个最近邻
#     [_, idx, _] = kdtree.search_knn_vector_3d(pc_load.points[i], k)
#     neighbors_normal_index[i] = idx  # 存储索引
        
#     # 提取近邻点的坐标
#     neighbor_points = origin_points[idx, :]
        
#     # 计算协方差矩阵
#     mean_vec = np.mean(neighbor_points, axis=0)
#     cov_mat = (neighbor_points - mean_vec).T @ (neighbor_points - mean_vec) / len(idx)
        
#     # 计算特征值和特征向量
#     eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)
        
#     # 最小特征值对应的向量是法向量
#     normal = eigen_vectors[:, np.argmin(eigen_values)]
#     normals.append(normal)

In [4]:
diffs = np.ones((20,3))
diffs = np.array([1,1,1,1])